In [ ]:
import pip
try:
    import pandas as pd
except ImportError:
        pip.main(['install', pandas])  
import os

from scipy.interpolate import interp1d

try:
    import numpy as np
except ImportError:
    pip.main(['install', numpy])  
    
try:
    import warnings
except ImportError:
        pip.main(['install', warnings])  

try:
    from openpyxl import load_workbook
except ImportError:
        pip.main(['install', openpyxl])  

try:
    import xlsxwriter
except ImportError:
        pip.main(['install', xlsxwriter])  
try:
    import matplotlib.pyplot as plt
except ImportError:
        pip.main(['install', matplotlib])  
        
from scipy.interpolate import interp1d
        
from datetime import datetime

try:
    from scipy.stats import t
except ImportError:
    pip.main(['install', scipy.stats]) 
    

warnings.filterwarnings("ignore")

working_directory = os.getcwd()
print(working_directory)
try:
    desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
except:
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop') 
print(desktop)

## Vacancy data

To compute net yields from gross yields, we need to know the percentage of rental homes that sit vacant.  We can get this information from the AHS as well.  We use the same dataset(including removing units in housing projects, those with bars on the windows, those thatare rent stabilized, and those missing data).  We label a home as a vacant rental if the survey identifies it as for rent only, for rent or for sale, or rented but not yet occupied.  The vacancy rate is the ratio of this number to this number plus the number of renter-occupied homes.For those city-year cells without enough datapoints, we use a projection from the rest of thedataset.

Response codes
01: For rent only
02: For rent or for sale
03: For sale only
04: Rented, but not yet occupied
05: Sold, but not yet occupied
06: Held for occasional use throughout the year
07: Other (specify)
08: Seasonal-Summer only
09: Seasonal-Winter only
10: Other seasonal (SPECIFY)
11: Migratory
N or -6: Not applicable



Response Codes that we will consider vacant

01: For rent only
02: For rent or for sale
03: For sale only
04: Rented, but not yet occupied

In [ ]:
def get_distribution(data, filter_code):
    dct = {}

    for i in data.year.unique():
            dct[i] = data.loc[data.year == i, filter_code].value_counts()

    dct = pd.DataFrame(dct)

    return dct

## Tax Rates

We also need a panel of tax rates to compute net yields.  Our sources are Emrath (2002) for1990 and 2000 tax rates from Census data, and the National Association of Home Builders(NAHB) for 2005 to 2012 tax rates from ACS data.  The tax rate data are available by state.

##  Interpolating missing years

As  the  survey  is  biannual  and  the  tax  rates  are  from  Census,  we  linearly  interpolate  therent-to-price ratios, vacancy rates, and tax rates to even-numbered years and other missingyears (in the case of the tax data)

## Net Yields

Starting from gross yields, we compute net yields using the follow costs, some of which areexpressed as a percentage of rent and some of which are a percentage of home value.  Weuse expense ratios from Morgan Stanley,  “The New Age of Buy-To-Rent,”  July 31,  2013.Similar, but less comprehensive, assumptions appear in Bernanke (2012) “The US HousingMarket:  Current conditions and policy considerations.”  The assumptions underlying CoreLogic’s Rental Trends, discussed below, are also broadly consistent with ours, however someof  their  cost  estimates  rely  on  direct  proprietary  data  rather  than  ratios  of  rent  or  houseprice.

- Insurance:  0.375% of price
- Repairs:  0.6% of price
- Capex:  1.15% of price
- Property manager:  5.9% of rent
- Credit loss:  0.73% of ren
- Tax: on price
- Vacancy:  on rent

get the rental data, vacancy data, and owned data

In [ ]:
def clean_outlier(hedonic_model_data, quant_remove, field):

    keep_ix = []
    for i in hedonic_model_data.CITY.unique():
        city_data = hedonic_model_data.loc[hedonic_model_data.CITY == i]
        lower_quant = city_data.loc[:,field].quantile(quant_remove)
        upper_quant = city_data.loc[:,field].quantile(1.0-quant_remove)
        city_data = city_data.loc[city_data[field] > lower_quant]
        city_data = city_data.loc[city_data[field] < upper_quant]
        keep_ix = keep_ix + city_data.index.tolist()
    return hedonic_model_data.loc[hedonic_model_data.index.isin(keep_ix)]

In [ ]:
rental_data = pd.read_pickle(os.path.join(desktop,'rented_cleaned.pkl'))
vacant = pd.read_pickle(os.path.join(desktop,'vacancy.pkl'))
owned_data = pd.read_pickle(os.path.join(desktop,'owned_cleaned_weighted.pkl'))



print(owned_data.columns)

In [ ]:
def get_taxes(data):
      
    data.loc[:,'states']=data.loc[:,'CITY']
    diction={'Anaheim':'California',
                                          'Atlanta': 'Georgia',
                                        'Baltimore':  'Maryland',
                                        'Boston':  'Massachusetts', 
                                        'Chicago': 'Illinois',
                                        'Cleveland':'Ohio',
                                          'Dallas':'Texas',
                                        'Detroit':'Michigan',
                                          'Houston':'Texas',
                                        'Kansas City': 'Missouri',
                                      'Los Angeles':'California',
                                          'Miami': 'Florida',
                                     'Minneapolis':'Minnesota',	
                                       'Nassau-Suffolk':'New York',
                                              'New York': 'New York',
                                            'Newark':'New Jersey',	
                                           'Oakland':'California',
                                    'Oklahoma City':'Oklahoma',
                                  'Philadelphia':'Pennsylvania',
                                           'Phoenix':'Arizona',
                                   'Pittsburgh':'Pennsylvania',
                                       'Riverside': 'California',
                                            'San Diego':'California',
                                           'San Francisco':'California',
                                            'San Jose':'California',
                                            'Washington':'DC',
                                         'Seattle':'Washington',
                                          'St. Louis':'Missouri',
                                              'Tampa':'Florida'}

    data['state']= data.states.map(diction)  
    data.loc[:, 'tax'] = 0.0


    tax_file=os.path.join(working_directory, 'tax_data.xlsx')
    tax_dta=pd.read_excel(tax_file)
    tax_dta.columns=['state_code','state',1990,2000,2009,2014]
    tax_dta.loc[:,[1990,2000,2009,2014]] = tax_dta.loc[:,[1990,2000,2009,2014]] / 1000.

    tax_dta.columns=['state_code','state','1990','2000','2009','2014']
    tax_complete=pd.DataFrame(np.nan,index=tax_dta.index, columns=range(1985,2018))
    

    # linear interpolation
    for i in range(tax_dta.shape[0]):
        x=[1990,2000,2009,2014]
        y= tax_dta.iloc[i,:][2:].values
        f=interp1d(x,y, fill_value='extrapolate')
        xnew=np.linspace(1985,2017,num=33,endpoint=True)
        tax_complete.iloc[i,:]=f(xnew)

    tax_complete.insert(0,'state_code',tax_dta.state_code)
    tax_complete.insert(1,'state',tax_dta.state)

    tax_complete.head(5)

    for i in tax_complete['state'].tolist():
        for j in tax_complete.columns[2:].tolist():
            data.loc[(data['state'] == i)&(data['year'] == j), 'tax'] = \
            tax_complete.loc[tax_complete['state'] == i,j].values[0]
    
    return data

function for helping to find the pooled data

In [ ]:
def get_pooled_data(net_yield_owned, string_var):
    net_yield_dict = {}


    for city in net_yield_owned['CITY'].unique():
        net_yield_dict[city] = {}
        for y in net_yield_owned['year'].unique():
            net_yield_dict[city][y] = net_yield_owned.loc[(net_yield_owned['CITY']== city) & (net_yield_owned['year']==y), string_var].mean()
    ny_df_pooled = pd.DataFrame.from_dict(net_yield_dict, orient = 'index')
    
    return pd.DataFrame(ny_df_pooled.dropna())

fixed costs

In [ ]:
insurance = 0.375 / 100.0
repairs = 0.6 / 100.0
capex = 1.15 / 100.0
property_manager = 5.9 / 100.0
credit_loss = 0.73 / 100.0 

fixed_rent_costs = property_manager + credit_loss
fixed_price_costs = insurance + repairs + capex 
print("fixed rent costs")
print(fixed_rent_costs)
print("fixed price costs")
print(fixed_price_costs)

owned_data.loc[:, 'fixed rent costs'] = fixed_rent_costs
owned_data.loc[:, 'fixed price costs'] = fixed_price_costs

get the vacancy costs. note the vacancy costs are on rent

In [ ]:
owned_data.loc[:, 'vacancy costs'] = 0.0

for i in vacant['CITY'].unique():
    print("The vacancy rate for " + str(i) + ' is')
    vacancy_rate = vacant.loc[vacant['CITY'] == i, 'Is_Vacant'].sum() / len(vacant.loc[vacant['CITY'] == i, 'Is_Vacant'])
    print(vacancy_rate)
    owned_data.loc[owned_data['CITY'] == i, 'vacancy costs'] = vacancy_rate
    
owned_data.loc[:, 'vacancy costs'].hist()

get the tax costs

In [ ]:
owned_data=owned_data.reset_index(drop = True)
rental_data=rental_data.reset_index(drop = True)

owned_data = get_taxes(owned_data)
rental_data = get_taxes(rental_data)
       
print(owned_data['tax'].hist())

In [ ]:
print(rental_data['tax'].hist())

get the total costs

In [ ]:
owned_data.loc[:, 'total costs'] = (owned_data.loc[:, 'fixed rent costs']  + owned_data.loc[:, 'vacancy costs']) * owned_data.loc[:,'RENT'] \
+ (owned_data.loc[:, 'tax'] + owned_data.loc[:, 'fixed price costs']) * owned_data.loc[:, 'VALUE']


#histogram of total costs
owned_data.loc[:, 'total costs'].hist()




In [ ]:

(owned_data.loc[:, 'total costs'] / owned_data.loc[:,'RENT']).hist()

appreciation

In [ ]:
appreciation_data = pd.read_csv(os.path.join(working_directory, 'appreciation_data/City_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv'))


change_data = appreciation_data[appreciation_data.columns[8:]]

change_data.columns = pd.to_datetime(change_data.columns)


appreciation_data.drop(labels = appreciation_data.columns[8:] , axis = 1, inplace = True)


rates = pd.DataFrame()

count = 1997

while count <= 2013:

    end = datetime(count+2, 6, 1)

    start = datetime(count, 6, 1)

    

    s = change_data[change_data.columns[(change_data.columns >= start)]]

    e = s[s.columns[(s.columns <= end)]]
            

    appreciation_data[count] = ((1.+((e[e.columns[-1]] - e[e.columns[0]]) / e[e.columns[0]]))**.5)-1.
    #appreciation_data[count] = ((e[e.columns[-1]] - e[e.columns[0]]) / e[e.columns[0]]) / 2.0
    
    #get the biweekly change

    count = count + 2

    

print(appreciation_data)


net_yield_owned = owned_data.loc[owned_data['year'] >= 1997]
net_yield_owned.loc[:, 'appreciation'] = 0.0
net_yield_owned = net_yield_owned.loc[net_yield_owned['CITY'] != 'Nassau-Suffolk']


error_cities = []

app_dict = {}

for city in net_yield_owned['CITY'].unique():
    app_dict[city] = {}

    for y in net_yield_owned['year'].unique():

        if len(appreciation_data.loc[(appreciation_data['RegionName'] == city.rstrip()), y]) >= 1:

            if len(net_yield_owned.loc[(net_yield_owned['CITY']== city) & (net_yield_owned['year']==y), 'appreciation']) >= 1:

                net_yield_owned.loc[(net_yield_owned['CITY']== city) & (net_yield_owned['year']==y), 'appreciation'] = appreciation_data.loc[(appreciation_data['RegionName'] == city.strip()), y].values[0]
                app_dict[city][y] = appreciation_data.loc[(appreciation_data['RegionName'] == city.strip()), y].values[0]
        else:

            error_cities.append([city, y])


print(error_cities)
print(net_yield_owned['CITY'].unique())

app_df = pd.DataFrame.from_dict(app_dict, orient = 'index')
print(app_df)


In [ ]:
net_yield_owned.loc[:, 'appreciation'].hist()

In [ ]:
print(len(net_yield_owned))
net_yield_owned = net_yield_owned.dropna(subset = ['VALUE', 'appreciation', 'total costs', 'RENT'], axis = 0, how = 'any')
print(len(net_yield_owned))

net_yield_owned.loc[:, 'total returns'] = (net_yield_owned .loc[:, 'VALUE'] * net_yield_owned.loc[:, 'appreciation'] + net_yield_owned.loc[:,'RENT'] - \
                                   net_yield_owned.loc[:, 'total costs']) / net_yield_owned .loc[:, 'VALUE']


net_yield_owned.loc[:, 'net yields'] = (net_yield_owned.loc[:,'RENT'] - \
                                   net_yield_owned.loc[:, 'total costs']) / net_yield_owned .loc[:, 'VALUE']

net_yield_owned.loc[:, 'net yields'].hist()

net_yield_owned.loc[:, 'gross yields'] = (net_yield_owned.loc[:,'RENT'] ) / net_yield_owned .loc[:, 'VALUE']


In [ ]:
net_yield_owned.loc[:,'RENT'].mean()
print(( net_yield_owned.loc[:, 'net yields'] / net_yield_owned.loc[:, 'gross yields']).mean())
print(( net_yield_owned.loc[:, 'net yields'] / net_yield_owned.loc[:, 'gross yields']).median())


In [ ]:
net_yields_by_year = {}

for i in net_yield_owned['year'].unique():
    net_yields_by_year[i] = net_yield_owned.loc[net_yield_owned['year'] == i, 'net yields'].mean()
    
net_yields_by_year = pd.DataFrame.from_dict(net_yields_by_year, orient = 'index')

net_yields_by_year.plot()

In [ ]:
#gross yields by year
gross_yields_by_year = {}

for i in net_yield_owned['year'].unique():
    gross_yields_by_year[i] = net_yield_owned.loc[net_yield_owned['year'] == i, 'gross yields'].mean()
    
gross_yields_by_year = pd.DataFrame.from_dict(gross_yields_by_year, orient = 'index')

gross_yields_by_year.plot()

In [ ]:
net_yield_owned.loc[:, 'gross yields'].mean()

get the quintile information

In [ ]:
quintiles = pd.DataFrame(pd.qcut(net_yield_owned['VALUE'], 5, labels=np.arange(5, 0, -1)))
quintiles.columns = ['quint']
quintiles = pd.concat([net_yield_owned[['net yields', 'appreciation', 'total returns']], quintiles], axis = 1)
print(quintiles)

quintile_dict = {}
for i in quintiles['quint'].unique():
    quintile_dict[i] = [quintiles.loc[quintiles['quint'] == i, 'appreciation'].mean(), \
                        quintiles.loc[quintiles['quint'] == i, 'net yields'].mean(), \
                        quintiles.loc[quintiles['quint'] == i, 'total returns'].mean()]
                        
    
quintile_dict = pd.DataFrame.from_dict(quintile_dict, orient = 'index')

quintile_dict.columns = ['appreciation', 'net yield', 'total returns']

print('mean')
print(quintile_dict.sort_values(by = ['net yield'], ascending = False))

quintile_dict = {}
for i in quintiles['quint'].unique():
    quintile_dict[i] = [quintiles.loc[quintiles['quint'] == i, 'appreciation'].median(), \
                        quintiles.loc[quintiles['quint'] == i, 'net yields'].median(), \
                        quintiles.loc[quintiles['quint'] == i, 'total returns'].median()]
                        
    
quintile_dict = pd.DataFrame.from_dict(quintile_dict, orient = 'index')

quintile_dict.columns = ['appreciation', 'net yield', 'total returns']

print('median')
print(quintile_dict.sort_values(by = ['net yield'], ascending = False))


quintile_dict = {}
for i in quintiles['quint'].unique():
    quintile_dict[i] = [quintiles.loc[quintiles['quint'] == i, 'appreciation'].std(), \
                        quintiles.loc[quintiles['quint'] == i, 'net yields'].std(), \
                        quintiles.loc[quintiles['quint'] == i, 'total returns'].std()]
                        
    
quintile_dict = pd.DataFrame.from_dict(quintile_dict, orient = 'index')

quintile_dict.columns = ['appreciation', 'net yield', 'total returns']

print('std')
print(quintile_dict.sort_values(by = ['net yield'], ascending = False))

In [ ]:
net_yield_dict = {}


for city in net_yield_owned['CITY'].unique():
    net_yield_dict[city] = {}
    for y in net_yield_owned['year'].unique():
        net_yield_dict[city][y] = net_yield_owned.loc[(net_yield_owned['CITY']== city) & (net_yield_owned['year']==y), 'net yields'].mean()
ny_df_pooled = pd.DataFrame.from_dict(net_yield_dict, orient = 'index')

print("Net Yield By City By Year")
print(pd.DataFrame(ny_df_pooled.dropna()))
pd.DataFrame(ny_df_pooled.dropna()).to_csv('net_yields_city_year.csv')
csv_r = pd.DataFrame(ny_df_pooled.dropna().mean(axis = 1))
print("Mean Net Yield By City Pooled Over City")
print(csv_r)
print("Mean Net Yield Pooled Over Year")
print(pd.DataFrame(ny_df_pooled.dropna().mean(axis = 0)))

csv_r.columns = ['net yields']
csv_r.to_csv('net_yields_city.csv')

In [ ]:
import seaborn as sns
ax = sns.heatmap(csv_r.sort_values(by = csv_r.columns[0]),annot=True)

In [ ]:
ax = sns.heatmap(pd.DataFrame(ny_df_pooled.dropna().mean(axis = 0)),annot=True)

In [ ]:
ax = sns.heatmap(pd.DataFrame(ny_df_pooled.dropna()),cmap="Reds")

In [ ]:
geocodes = pd.read_excel('fip_codes.xls',encoding="utf-8",dtype = str)
geocodes['FIPS'] = ''
geocodes['FIPS'] = geocodes['FIPS State'].map(str) + geocodes['FIPS County'].map(str)
geocodes['County Name'] = geocodes['County Name'].map(lambda x: x.replace('St ', 'St. '))
print(geocodes.head(10))

In [ ]:
df = get_pooled_data(net_yield_owned, 'net yields')
print(df)

print(' ')
print("Net Yield City Level")
df = pd.DataFrame(df.mean(axis = 1))
df.columns = ['net yield' ]
print(df)


### Net Yields 

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px


mapping = pd.read_csv('city_county_mapping.csv', index_col = 0)
df = pd.concat([df, mapping], axis = 1)

for ix, row in df.iterrows():
    state = net_yield_owned.loc[net_yield_owned.CITY == ix, 'state'].unique()[0]
    if state == 'DC':
        state = 'District of Columbia'
    df.loc[ix,'FIPS'] = geocodes.loc[(row.county.strip() == geocodes['County Name']) & (state == geocodes['State'])]['FIPS'].values[0]


fig = px.choropleth(df, geojson=counties, locations='FIPS', color='net yield',
                           color_continuous_scale="Rainbow",
                           range_color=(df['net yield'].min()-.01,df['net yield'].max()+.01),
                           scope="usa"
                          )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Appreciation Data

In [ ]:
app_df_pooled = get_pooled_data(net_yield_owned, 'appreciation')
print(app_df_pooled)

print(' ')
print("Appreciation City Level")
app_df_pooled = pd.DataFrame(app_df_pooled.mean(axis = 1))
app_df_pooled.columns = ['appreciation' ]
print(app_df_pooled)


In [ ]:
mapping = pd.read_csv('city_county_mapping.csv', index_col = 0)
app_df_pooled = pd.concat([app_df_pooled, mapping], axis = 1)

for ix, row in app_df_pooled.iterrows():
    state = net_yield_owned.loc[net_yield_owned.CITY == ix, 'state'].unique()[0]
    if state == 'DC':
        state = 'District of Columbia'
    app_df_pooled.loc[ix,'FIPS'] = geocodes.loc[(row.county.strip() == geocodes['County Name']) & (state == geocodes['State'])]['FIPS'].values[0]

fig = px.choropleth(app_df_pooled, geojson=counties, locations='FIPS', color='appreciation',
                           color_continuous_scale="Rainbow",
                           range_color=(app_df_pooled['appreciation'].min()-.01,app_df_pooled['appreciation'].max()+.01),
                           scope="usa"
                          )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Net Yields 2007-2009

In [ ]:

plot_city = pd.read_csv('net_yields_2007_county.csv')
plot_city.columns = ['city','net yield','county','fips_manual']
print(plot_city.head(10))
plot_city['FIPS'] = ''

for ix, row in plot_city.iterrows():
    state = net_yield_owned.loc[net_yield_owned.CITY == row.city, 'state'].unique()[0]
    if state == 'DC':
        state = 'District of Columbia'
    plot_city.loc[plot_city.index == ix,'FIPS'] = geocodes.loc[(row.county.strip() == geocodes['County Name']) & (state == geocodes['State'])]['FIPS'].values[0]

fig = px.choropleth(plot_city, geojson=counties, locations='FIPS', color='net yield',
                           color_continuous_scale="Rainbow",
                           range_color=(plot_city['net yield'].min()-.01,plot_city['net yield'].max()+.01),
                           scope="usa"
                          )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Total Returns

In [ ]:

total_returns_df_pooled = get_pooled_data(net_yield_owned, 'total returns')
print(total_returns_df_pooled)

print("Total Returns House Level")
city_total_returns = pd.DataFrame(total_returns_df_pooled.mean(axis = 1))
city_total_returns.columns = ['total returns' ]
print(city_total_returns)




In [ ]:

mapping = pd.read_csv('city_county_mapping.csv', index_col = 0)
city_total_returns = pd.concat([city_total_returns, mapping], axis = 1)

for ix, row in city_total_returns.iterrows():
    state = net_yield_owned.loc[net_yield_owned.CITY == ix, 'state'].unique()[0]
    if state == 'DC':
        state = 'District of Columbia'
    city_total_returns.loc[ix,'FIPS'] = geocodes.loc[(row.county.strip() == geocodes['County Name']) & (state == geocodes['State'])]['FIPS'].values[0]

fig = px.choropleth(city_total_returns, geojson=counties, locations='FIPS', color='total returns',
                           color_continuous_scale="Rainbow",
                           range_color=(city_total_returns['total returns'].min()-.01,city_total_returns['total returns'].max()+.01),
                           scope="usa"
                          )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from IPython.display import Image
Image("table3.png")

## Replicaction of Table 2 p. 44 of pdf

In [ ]:
table2 = pd.concat([city_total_returns,app_df_pooled,df ], axis = 1)[['total returns', 'net yield', 'appreciation']]

table2 = table2.sort_values(by = ['total returns'], ascending = False)
print(table2)

print('rankings')
ranking = table2.rank(axis = 0, ascending = False)
ranking['total returns'] = table2['total returns']
print(ranking)

## Replicaction of Table 3 p. 44 of pdf

In [ ]:
df = get_pooled_data(net_yield_owned, 'net yields')
table3 = table2.copy()
std = pd.DataFrame(total_returns_df_pooled.std(axis = 1)) * 100.0
std.columns = ['Std Total Returns']
std_ny = pd.DataFrame(df.std(axis = 1))* 100.0
std_ny.columns = ['Std Net Yields']

sharpe = pd.DataFrame(total_returns_df_pooled.mean(axis = 1) / total_returns_df_pooled.std(axis = 1))
sharpe.columns= ['Sharpe Total Returns']
sharpe_ny = pd.DataFrame(df.mean(axis = 1) / df.std(axis = 1))
sharpe_ny.columns= ['Sharpe Net Yields']

table3 = pd.concat([table3, sharpe, std, sharpe_ny, std_ny], axis = 1)
table3['% of Total Return from Net Yield'] = (table3['net yield'] / table3['total returns'])*100.0

print(table3)

### Applications to AssetPricing

In [ ]:
print(total_returns_df_pooled.head(5))

equal_weighted = total_returns_df_pooled.mean(axis = 0)
equal_weighted = pd.DataFrame(equal_weighted)
equal_weighted.columns = ['equal-weighted']
equal_weighted.index = pd.to_datetime(equal_weighted.index, format= '%Y')
equal_weighted.cumsum().plot()

In [ ]:
factors = pd.read_csv(os.path.join(working_directory,r'F-F_Research_Data_Factors.CSV'), skiprows = 3, index_col=0)
factors = factors.dropna(how = 'any', axis = 0)
factors = factors.apply(lambda x: x/ 100)
factors.index = pd.to_datetime(factors.index, format= '%Y')
print(factors.head(10))

In [ ]:
import statsmodels.api as sm
asset_pricing = pd.concat([equal_weighted, factors], axis = 1)
asset_pricing = asset_pricing.loc[(asset_pricing.index >= equal_weighted.index[0]) & (asset_pricing.index <= equal_weighted.index[-1]), :]
#forward fill
asset_pricing = asset_pricing.fillna(method='ffill')

Y = asset_pricing['equal-weighted']
X = asset_pricing[['Mkt-RF',    'SMB',    'HML',    'RF']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

### multi bandit optimization

In [ ]:
import random
state_returns = total_returns_df_pooled.T
print(state_returns.head(5))

epsilon = .25
backtest = {}
Q = {}
k = {}

symbols = list(state_returns.columns)

count = 0
for ix, row in state_returns.iterrows():
    if count >= 1:
        sortedQ = sorted(Q.items(), key=lambda kv: kv[1], reverse = True)
        if np.random.uniform() >= epsilon:
            k[sortedQ[0][0]] = k[sortedQ[0][0]] + 1
            Q[sortedQ[0][0]] = Q[sortedQ[0][0]] + (1./k[sortedQ[0][0]])*(row[sortedQ[0][0]] - Q[sortedQ[0][0]])
            backtest[ix] = row[sortedQ[0][0]]
            
        else:
            other_choices = symbols.copy()
            other_choices.remove(sortedQ[0][0])
            random_choice = random.choice(other_choices)
            k[random_choice] = k[random_choice] + 1
            Q[random_choice] = Q[random_choice] + (1./k[random_choice])*(row[random_choice] - Q[random_choice])
            backtest[ix] = row[random_choice]
        
    else:
        for i in state_returns.columns:
            Q[i] = row[i]
            k[i] = 1.
            
    count = count + 1
    
backtest = pd.DataFrame.from_dict(backtest, orient = 'index')
backtest = pd.DataFrame(backtest)
backtest.columns = ['epsilon greedy']

cum_backtest = backtest.cumsum()
cum_backtest.plot()


In [ ]:
equal_weighted = total_returns_df_pooled.mean(axis = 0)
equal_weighted = pd.DataFrame(equal_weighted)

ix_val = cum_backtest.index.values

plt.figure()
plt.plot(ix_val , equal_weighted.cumsum().loc[ix_val,:], label = 'equal-weighted')
plt.plot(ix_val , cum_backtest.loc[ix_val,:], label = 'epsilon greedy portfolios')

plt.legend(loc='upper right')
plt.title('Test')
plt.show()